In [1]:
using RigidBodyDynamics
using RigidBodyDynamics.TreeDataStructure
using RigidBodyDynamics.OdeIntegrators
using RigidBodyTreeInspector
using VariableHeightMomentumControl
using JuMP
using Gurobi
using StaticArrays

import RigidBodyDynamics: @framecheck

adding: /Users/twan/code/julia/RigidBodyDynamics/v0.5/DrakeVisualizer/src/lcmtypes to the python path


In [2]:
# TODO: grab everything off of github and cache
DRAKE_DISTRO = "$(ENV["HOME"])/code/drake-distro"
urdf = "$DRAKE_DISTRO/drake/examples/Valkyrie/urdf/urdf/valkyrie_A_sim_drake_one_neck_dof_wide_ankle_rom.urdf"
mechanism = RigidBodyDynamics.parse_urdf(Float64, urdf);

In [3]:
# Create visualizer
package_path = ["$DRAKE_DISTRO/drake/examples"]
vis = parse_urdf(urdf, mechanism; package_path = package_path);

In [4]:
# Add sole frame
ankleRoll = edge_to_parent_data(findfirst(v -> !isroot(v) && edge_to_parent_data(v).name == "leftAnkleRoll", tree(mechanism)))
soleFrame = CartesianFrame3D("l_foot_sole")
soleToFoot = Transform3D(soleFrame, ankleRoll.frameAfter, SVector(0.067, 0., -0.09))
add_body_fixed_frame!(mechanism, soleToFoot);

In [5]:
# Reroot at the left foot and remove fixed joints
pelvis = vertex_data(findfirst(v -> name(vertex_data(v)) == "pelvis", tree(mechanism)))
leftFoot = vertex_data(findfirst(v -> name(vertex_data(v)) == "leftFoot", tree(mechanism)))
leftFootToWorld = Joint("", Fixed{Float64}())
world = root_body(mechanism)
reattach!(mechanism, pelvis, 
    world, leftFootToWorld, Transform3D(leftFootToWorld.frameBefore, world.frame, -soleToFoot.trans), 
    leftFoot, Transform3D{Float64}(leftFoot.frame, leftFootToWorld.frameAfter))
remove_fixed_joints!(mechanism);

In [6]:
type VariableHeightMomentumController
    τ::Vector{Float64}
    dynamicsResult::DynamicsResult{Float64}
    positionControlledJoints::Vector{Joint{Float64}}
    
    function VariableHeightMomentumController(mechanism::Mechanism, positionControlledJoints::Vector{Joint{Float64}})
        τ = zeros(num_velocities(mechanism))
        new(τ, DynamicsResult(Float64, mechanism), positionControlledJoints)
    end
end

In [7]:
# assumes that the plane in which the CoP lies is the x-y plane of the frame in which the wrench is expressed
function center_of_pressure(wrench::Wrench)
    T = eltype(wrench)
    normal = StaticArrays.SVector(zero(T), zero(T), one(T))
    torque = wrench.angular
    force = wrench.linear
    Point3D(wrench.frame, normal × torque / (normal ⋅ force))
end

center_of_pressure (generic function with 1 method)

In [8]:
function cubic_orbital_energy_controller(g, zf, x, z, xd, zd)
    a = xd / x
    b = zd - a * z
    u = -7 * a^2 + (3 * zf * a^3 - g * a) / b - (10 * a^3 * b) / g
end

cubic_orbital_energy_controller (generic function with 1 method)

In [9]:
const centroidalFrame = CartesianFrame3D("centroidal")

CartesianFrame3D: "centroidal" (id = 357)

In [10]:
function control(controller::VariableHeightMomentumController, t, state)
    # create model
    solver = Gurobi.GurobiSolver()
    Gurobi.setparameters!(solver, Silent = true)
    model = Model(solver = solver)
    nv = num_velocities(state)
    @variable(model, v̇[1 : nv])
    
    # Compute desired rate of change of linear momentum
    com = center_of_mass(state) - transform(state, Point3D(Float64, soleFrame), root_frame(mechanism))
    v = velocity_vector(state)
    A = momentum_matrix(state)
    m = mass(state.mechanism)
    comdot = FreeVector3D(A.frame, A.linear * v) / m
    g = -RigidBodyDynamics.gravitational_spatial_acceleration(state.mechanism).linear[3]
    zf = 0.95
    x, z = com.v[1], com.v[3]
    xd, zd = comdot.v[1], comdot.v[3]
    u = cubic_orbital_energy_controller(g, zf, x, z, xd, zd)
    u = max(u, zero(u))
    fx = m * x * u
    fz = m * (z * u - g)
    ḣdes = Wrench{Float64}(centroidalFrame, zero(SVector{3, eltype(u)}), SVector(fx, zero(u), fz))
    centroidalToWorld = Transform3D(centroidalFrame, root_frame(mechanism), com.v)
    ḣdes = transform(ḣdes, centroidalToWorld)
    
    # Set up QP
    Ȧv = momentum_rate_bias(state)
    @framecheck A.frame ḣdes.frame
    @framecheck A.frame Ȧv.frame
    @constraint(model, Array(A.linear) * v̇ + Array(Ȧv.linear) .== Array(ḣdes.linear))
    @constraint(model, Array(A.angular) * v̇ + Array(Ȧv.angular) .== Array(ḣdes.angular))
    
    # PD control for upper body and free leg
    kp = 100.
    kd = 20.
    q = configuration_vector(state)
    v = velocity_vector
    for joint in positionControlledJoints
        v̇pd = -kp .* configuration(state, joint) - kd .* velocity(state, joint)
        @constraint(model, v̇[mechanism.vRanges[joint]] .== v̇pd)
    end
    
    # minimize squared joint accelerations
    @objective(model, Min, sum{v̇[i]^2, i = 1 : nv})
    
    # solve
    status = solve(model)
    
    # Compute CoP
    gravitationalWrench = transform(Wrench(centroidalFrame, zero(SVector{3}), SVector(0, 0, -m * g)), centroidalToWorld)
    groundReactionWrench = ḣdes - gravitationalWrench
    groundReactionWrench = transform(state, groundReactionWrench, soleFrame)
    cop = center_of_pressure(groundReactionWrench)
    cop = transform(state, cop, root_frame(mechanism))
    @assert all(isnan(cop.v)) || isapprox(cop.v[1], 0., atol = 1e-8)

    # inverse dynamics
    result = controller.dynamicsResult
    result.v̇ = getvalue(v̇)
    inverse_dynamics!(controller.τ, result.jointWrenches, result.accelerations, state, result.v̇)
    controller.τ
end

control (generic function with 1 method)

In [11]:
pelvisVertex = findfirst(v -> name(vertex_data(v)) == "pelvis", tree(mechanism))
stanceFootVertex = findfirst(v -> name(vertex_data(v)) == "world", tree(mechanism))
stanceLegPath = path(pelvisVertex, stanceFootVertex)
positionControlledJoints = collect(filter(j -> j ∉ stanceLegPath.edgeData, joints(mechanism)))
const controller = VariableHeightMomentumController(mechanism, positionControlledJoints);

In [12]:
function passive_dynamics!(vd::AbstractArray, t, state)
    dynamics!(result, state)
    copy!(vd, result.v̇)
end

passive_dynamics! (generic function with 1 method)

In [13]:
function controlled_dynamics!(vd::AbstractArray, t, state)
    torques = control(controller, t, state)
    dynamics!(result, state, torques)
    copy!(vd, result.v̇)
end

controlled_dynamics! (generic function with 1 method)

In [14]:
# Simulation
const state = MechanismState(Float64, mechanism)
const result = DynamicsResult(Float64, mechanism)
const integrator = MuntheKaasIntegrator(controlled_dynamics!, runge_kutta_4(Float64), DrakeVisualizerSink(vis, 1e-2));

In [15]:
ankleRoll = edge_to_parent_data(findfirst(v -> !isroot(v) && edge_to_parent_data(v).name == "leftAnkleRoll_flipped", tree(mechanism)))
knee = edge_to_parent_data(findfirst(v -> !isroot(v) && edge_to_parent_data(v).name == "leftKneePitch_flipped", tree(mechanism)))
hipPitch = edge_to_parent_data(findfirst(v -> !isroot(v) && edge_to_parent_data(v).name == "leftHipPitch_flipped", tree(mechanism)))
hipRoll = edge_to_parent_data(findfirst(v -> !isroot(v) && edge_to_parent_data(v).name == "leftHipRoll_flipped", tree(mechanism)))

Joint "leftHipRoll_flipped": Revolute joint with axis [-1.0,-0.0,-0.0]

In [16]:
function solve_for_velocities!(state::MechanismState, momentum::Momentum, fixedJoints::Vector{Joint{Float64}})
    solver = Gurobi.GurobiSolver()
    Gurobi.setparameters!(solver, Silent = true)
    model = Model(solver = solver)
    nv = num_velocities(state)
    A = momentum_matrix(state)
    
    momentum = transform(state, momentum, A.frame)
    
    @variable(model, v[1 : nv])
    @constraint(model, Array(A.linear) * v .== Array(momentum.linear))
    @constraint(model, Array(A.angular) * v .== Array(momentum.angular))
    for joint in fixedJoints
        @constraint(model, v[mechanism.vRanges[joint]] .== 0)
    end
    @objective(model, Min, sum{v[i]^2, i = 1 : nv})

    status = solve(model)
    set_velocity!(state, getvalue(v))
end

solve_for_velocities! (generic function with 1 method)

In [17]:
function initialize_state!(state::MechanismState, xd::Float64)
    zero!(state)
    θroll = 0.13
    θknee = 1.
    configuration(state, ankleRoll)[:] = θroll
    configuration(state, hipRoll)[:] = -θroll
    configuration(state, knee)[:] = θknee
    configuration(state, hipPitch)[:] = -θknee
    setdirty!(state)
    com = center_of_mass(state)
    comdot = FreeVector3D(centroidalFrame, SVector(xd, 0.0, 0.0))
    linearMomentum = mass(mechanism) * comdot
    initialMomentum = Momentum{Float64}(centroidalFrame, zero(SVector{3}), linearMomentum.v)
    centroidalToWorld = Transform3D(centroidalFrame, root_frame(mechanism), com.v)
    initialMomentum = transform(initialMomentum, centroidalToWorld)
    solve_for_velocities!(state, initialMomentum, positionControlledJoints)
end

initialize_state! (generic function with 1 method)

In [23]:
xd = 1.0 + (1.4 - 1.0) * rand()
initialize_state!(state, xd)
integrate(integrator, state, 1.5, 1e-3)
# control(controller, 0., state)